In [1]:
print("Importa librerie", end="", flush=True)
from keras_tuner import HyperModel, RandomSearch, Hyperband, BayesianOptimization
print(".", end="", flush=True)
from tensorflow.keras.layers import LSTM, RepeatVector, Dropout, TimeDistributed, Dense
print(".", end="", flush=True)
from tensorflow.keras.models import Sequential
print(".", end="", flush=True)
from tensorflow.keras.regularizers import l2
print(".", end="", flush=True)
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
print(".", end="", flush=True)
import pandas as pd
print(".", end="", flush=True)
import yfinance as yf
print(".", end="", flush=True)
import numpy as np
print(".", end="", flush=True)
import tensorflow as tf
print(".", end="", flush=True)
import funzioni as fx
print(".", end="", flush=True)
from sklearn.preprocessing import PowerTransformer
print(".", end="", flush=True)
from sklearn.base import clone
print(".", end="", flush=True)
import os
print("=", end="\n", flush=True)


Importa librerie............=


In [2]:
simbolo_test = "BTG"
simbolo_validazione = "DHT"
n_simboli_addestramento = 1000
n_timesteps = 60 # n. barre del periodo passato per la ricerca di pattern, inclusa ultima data disponibile
giorni_previsione = 10 # n. barre nel futuro di cui si desidera prevedere il prezzo

features_prezzo = [
    "Close",
    "EMA_5", 
    "EMA_20", 
    "EMA_50",
    "Open",  
    "High",
    "Low",
    "PSAR",
    "SUPERT", 
]

features_da_scalare_singolarmente = [
    "Volume",
    "ATR",
    "PSARaf",
    "ADX",
]

features_meno_piu = [
    "MACDh",    
    "AROONOSC",
    "TRIX",
    "TRIXs",
    "DM_OSC",
]

features_no_scala = [
    "SUPERTd",  
    "PSARr"
]

features_candele = [
    "CDL_2CROWS", "CDL_3BLACKCROWS", "CDL_3INSIDE", "CDL_3LINESTRIKE", "CDL_3OUTSIDE", "CDL_3STARSINSOUTH", "CDL_3WHITESOLDIERS", "CDL_ABANDONEDBABY", "CDL_ADVANCEBLOCK", "CDL_BELTHOLD", "CDL_BREAKAWAY", "CDL_CLOSINGMARUBOZU", "CDL_CONCEALBABYSWALL", "CDL_COUNTERATTACK", "CDL_DARKCLOUDCOVER", "CDL_DOJI_10_0.1", "CDL_DOJISTAR", "CDL_DRAGONFLYDOJI", "CDL_ENGULFING", "CDL_EVENINGDOJISTAR", "CDL_EVENINGSTAR", "CDL_GAPSIDESIDEWHITE", "CDL_GRAVESTONEDOJI", "CDL_HAMMER", "CDL_HANGINGMAN", "CDL_HARAMI", "CDL_HARAMICROSS", "CDL_HIGHWAVE", "CDL_HIKKAKE", "CDL_HIKKAKEMOD", "CDL_HOMINGPIGEON", "CDL_IDENTICAL3CROWS", "CDL_INNECK", "CDL_INSIDE", "CDL_INVERTEDHAMMER", "CDL_KICKING", "CDL_KICKINGBYLENGTH", "CDL_LADDERBOTTOM", "CDL_LONGLEGGEDDOJI", "CDL_LONGLINE", "CDL_MARUBOZU", "CDL_MATCHINGLOW", "CDL_MATHOLD", "CDL_MORNINGDOJISTAR", "CDL_MORNINGSTAR", "CDL_ONNECK", "CDL_PIERCING", "CDL_RICKSHAWMAN", "CDL_RISEFALL3METHODS", "CDL_SEPARATINGLINES", "CDL_SHOOTINGSTAR", "CDL_SHORTLINE", "CDL_SPINNINGTOP", "CDL_STALLEDPATTERN", "CDL_STICKSANDWICH", "CDL_TAKURI", "CDL_TASUKIGAP", "CDL_THRUSTING", "CDL_TRISTAR", "CDL_UNIQUE3RIVER", "CDL_UPSIDEGAP2CROWS", "CDL_XSIDEGAP3METHODS",
]

elenco_targets = [
    "EMA_5",
    "EMA_20", 
    "EMA_50"    
]

col_features_prezzo = {col: idx for idx, col in enumerate(features_prezzo)}
col_features_da_scalare_singolarmente = {col: idx for idx, col in enumerate(features_da_scalare_singolarmente)}
col_features_meno_piu = {col: idx for idx, col in enumerate(features_meno_piu)}
col_features_no_scala = {col: idx for idx, col in enumerate(features_no_scala)}
col_features_candele = {col: idx for idx, col in enumerate(features_candele)}
col_targets = {col: idx for idx, col in enumerate(elenco_targets)}
n_features = len(col_features_prezzo) + len(col_features_da_scalare_singolarmente) + len(col_features_meno_piu) + len(col_features_no_scala) + len(col_features_candele) 
n_targets = len(col_targets) 

class MixedHyperModel(HyperModel):
    def __init__(self, n_timesteps, n_features, n_targets, giorni_previsione):
        self.n_timesteps = n_timesteps
        self.n_features = n_features
        self.n_targets = n_targets
        self.giorni_previsione = giorni_previsione

    def build(self, hp):
        model = Sequential()

        # Layer LSTM iniziale
        model.add(LSTM(hp.Int('lstm_units_1', 20, 300, step=20),
                       input_shape=(self.n_timesteps, self.n_features)))
        model.add(RepeatVector(self.giorni_previsione))

        # Aggiunta di layer LSTM intermedi
        for i in range(hp.Int('num_lstm_layers', 1, 4)):
            model.add(LSTM(hp.Int(f'lstm_units_{i+2}', 20, 300, step=20), return_sequences=True,
                           kernel_regularizer=l2(hp.Float('l2_rate', 0, 5, sampling='log'))))
            model.add(Dropout(hp.Float(f'lstm_dropout_{i+2}', 0, 0.5, step=0.1)))

        # Aggiunta di layer Dense
        for i in range(hp.Int('num_dense_layers', 1, 4)):
            model.add(TimeDistributed(Dense(hp.Int(f'dense_units_{i}', 20, 300, step=20), activation='relu')))
            model.add(Dropout(hp.Float(f'dense_dropout_{i}', 0, 0.5, step=0.1)))

        model.add(TimeDistributed(Dense(self.n_targets, activation='linear')))

        model.compile(optimizer="adam",
                      loss='mean_absolute_percentage_error')

        return model

input_shape = (n_timesteps, n_features)
output_shape = (giorni_previsione, n_targets)
hypermodel = MixedHyperModel(n_timesteps, n_features, n_targets, giorni_previsione)



In [ ]:
if os.path.exists("lista_ticker.parquet"):
    print("Caricamento lista_ticker esistente")
    lista_ticker = pd.read_parquet("lista_ticker.parquet")
else:
    print("Download lista ticker")
    lista_ticker = pd.read_parquet("Tickers_De_Giro.parquet")
    lista_ticker = lista_ticker.sample(frac=1).reset_index(drop=True)
    lista_ticker = lista_ticker.loc[(lista_ticker["Ticker"] != simbolo_test) & (lista_ticker["Ticker"] != simbolo_validazione) & (lista_ticker["Categoria"] != "D"), :]

print("Download dati ticker validazione")
ticker_val = yf.download(simbolo_validazione, start='2010-01-01', end='2023-12-31', progress=False)
ticker_val.index = ticker_val.index.date
print("Calcolo indicatori ticker validazione")
ticker_val = fx.crea_indicatori(ticker_val)
ticker_val.dropna(axis=0, inplace=True)

idx_val, X_val, Y_val, _ = fx.to_XY(ticker_val, features_prezzo, features_da_scalare_singolarmente, features_meno_piu, features_candele, features_no_scala, elenco_targets, n_timesteps, giorni_previsione, addestramento=True)

if os.path.exists("X.npy") and os.path.exists("Y.npy"):
    X = np.load('X.npy')
    Y = np.load('Y.npy')
else:
    X = np.zeros((0, n_timesteps, n_features))
    Y = np.zeros((0, giorni_previsione, n_targets))
    
    
for i_ticker in range (n_simboli_addestramento):
    nome_simbolo = lista_ticker["Ticker"].iloc[i_ticker]
    print(f"\033[48;5;42m{i_ticker+1} di {n_simboli_addestramento}: Ticker {nome_simbolo}\033[0m")
    print("Download dati ticker")
    try:
        ticker = yf.download(nome_simbolo, start='2010-01-01', end='2023-12-31', progress=False)
        if ticker["Close"].iloc[-1] >= 1:
            ticker.index = ticker.index.date
            print("Calcolo indicatori ticker")
            ticker = fx.crea_indicatori(ticker)
            ticker.dropna(axis=0, inplace=True)

            print("Definizione features e target")
            idx, X_train, Y_train, _ = fx.to_XY(ticker, features_prezzo, features_da_scalare_singolarmente, features_meno_piu, features_candele, features_no_scala, elenco_targets, n_timesteps, giorni_previsione, addestramento=True)

            X = np.vstack((X, X_train))        
            Y = np.vstack((Y, Y_train))
            print("Salvataggio X e Y su file")
            np.save('X', X)
            np.save('Y', Y)
except Exception as e:
        print(e)
        continue


Download lista ticker
Download dati ticker validazione
Calcolo indicatori ticker validazione



1 Failed download:
['GVS']: Exception("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")


1 di 1000: Ticker GVS
Download dati ticker
single positional indexer is out-of-bounds
2 di 1000: Ticker PDS
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
3 di 1000: Ticker CNA
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
4 di 1000: Ticker PFG
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
5 di 1000: Ticker X
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
6 di 1000: Ticker HAS
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
7 di 1000: Ticker MOFG
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
8 di 1000: Ticker CRS
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
9 di 1000: Ticker NJR
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
10 di 1000: Ticker ALGM
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
11 di 1000: Ticker DK



1 Failed download:
['SLG^PRI']: Exception('%ticker%: No timezone found, symbol may be delisted')


13 di 1000: Ticker SLG^PRI
Download dati ticker
single positional indexer is out-of-bounds
14 di 1000: Ticker SXC
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
15 di 1000: Ticker SYF
Download dati ticker
Calcolo indicatori ticker
Definizione features e target


...


1 Failed download:
['1TEF']: Exception('%ticker%: No timezone found, symbol may be delisted')


single positional indexer is out-of-bounds
73 di 1000: Ticker PPG
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
74 di 1000: Ticker AMBC
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
75 di 1000: Ticker LNN
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
76 di 1000: Ticker PK
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
77 di 1000: Ticker WTW
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
78 di 1000: Ticker GCMG
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
79 di 1000: Ticker TOWN
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
80 di 1000: Ticker ON
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
81 di 1000: Ticker IVR
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
82 di 1000: Ticker MMSI
Download dati ticker
Calcolo ind


1 Failed download:
['BRK^B']: Exception('%ticker%: No timezone found, symbol may be delisted')


single positional indexer is out-of-bounds
90 di 1000: Ticker ABG
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
91 di 1000: Ticker FLS
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
92 di 1000: Ticker ENOV
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
93 di 1000: Ticker PRGS
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
94 di 1000: Ticker INFA
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
95 di 1000: Ticker LSXMK
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
96 di 1000: Ticker XPRO
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
97 di 1000: Ticker ALKS
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
98 di 1000: Ticker AGM
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
99 di 1000: Ticker TTMI
Download dati ticker
Calco

Calcolo indicatori ticker
Definizione features e target
162 di 1000: Ticker EXTR
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
163 di 1000: Ticker SPT
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
164 di 1000: Ticker EDN
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Unable to allocate 15.7 GiB for an array with shape (427552, 60, 82) and data type float64
165 di 1000: Ticker DAN
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Unable to allocate 15.7 GiB for an array with shape (427552, 60, 82) and data type float64
166 di 1000: Ticker GM
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
Unable to allocate 15.7 GiB for an array with shape (427330, 60, 82) and data type float64
167 di 1000: Ticker CRC
Download dati ticker
Calcolo indicatori ticker
Definizione features e target
168 di 1000: Ticker FHI
Download dati ticker
Calcolo indicatori t

In [ ]:
def rd(hypermodel, X_train, Y_train, X_val, Y_val):
    random_tuner = RandomSearch(
        hypermodel,
        objective='val_loss',
        max_trials=50,
        executions_per_trial=2,
        directory='random_search',
        project_name='random_tuning'
    )
    random_tuner.search_space_summary()
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    random_tuner.search(X_train, Y_train, epochs=epochs, validation_data=(X_val, Y_val), callbacks=[early_stopping, reduce_lr])
    # Ottieni il miglior modello e i migliori iperparametri
    best_model_random = random_tuner.get_best_models(num_models=1)[0]
    best_hyperparameters_random = random_tuner.get_best_hyperparameters(num_trials=1)[0]

    print("Migliori iperparametri per Random Search:", best_hyperparameters_random.values)
    
    return best_model_random, best_hyperparameters_random

In [ ]:
def bay(hypermodel, X_train, Y_train, X_val, Y_val):
    bayesian_tuner = BayesianOptimization(
        hypermodel,
        objective='val_loss',
        max_trials=50,
        directory='bayesian_search',
        project_name='bayesian_tuning'
    )
    
    bayesian_tuner.search_space_summary()
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    bayesian_tuner.search(X_train, Y_train, epochs=100, validation_data=(X_val, Y_val), callbacks=[early_stopping, reduce_lr])
    
    best_model_bayesian = bayesian_tuner.get_best_models(num_models=1)[0]
    best_hyperparameters_bayesian = bayesian_tuner.get_best_hyperparameters(num_trials=1)[0]

    print("Migliori iperparametri per Bayesian Optimization:", best_hyperparameters_bayesian.values)
    
    return best_model_bayesian, best_hyperparameters_bayesian


In [ ]:
def hb(hypermodel, X_train, Y_train, X_val, Y_val):
    hyperband_tuner = Hyperband(
        hypermodel,
        objective='val_loss',
        max_epochs=30,
        seed=seed,
        directory='hyperband_search',
        project_name='hyperband_tuning'
    )
    
    hyperband_tuner.search_space_summary()
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    hyperband_tuner.search(X_train, Y_train, validation_data=(X_val, Y_val), callbacks=[early_stopping, reduce_lr])
    
    best_model_hyperband = hyperband_tuner.get_best_models(num_models=1)[0]
    best_hyperparameters_hyperband = hyperband_tuner.get_best_hyperparameters(num_trials=1)[0]

    print("Migliori iperparametri per Hyperband:", best_hyperparameters_hyperband.values)
    
    return best_model_hyperband, best_hyperparameters_hyperband


In [ ]:
#model, par = rd(hypermodel, X, Y, X_val, Y_val)

In [ ]:
model, par = bay(hypermodel, X, Y, X_val, Y_val)